Con este programa generamos un dataframe con datos de imagenes

In [1]:
import os

home = os.path.expanduser("~")

db_name=  'colab_main_dataset' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
#output_dir = os.path.join(home, 'drive', '.shared','LR_face_video')
database_names=['enfsi','lfw','forenface'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["QMagFace"] # , "Dlib", "ArcFace",
attributes_to_update=[]  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

In [2]:
from sql_face.sqldb import SQLDataBase

database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

2023-05-15 20:18:18.230017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
#session.close()


In [6]:


import pandas as pd
import numpy as np
from tqdm import tqdm
from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage, Image, QualityModel, QualityGroup,Detector, EmbeddingModel

session = database.session

results = session.query(Image,Detector.name,EmbeddingModel.name,QualityModel.name,QualityImage.quality_group) \
                        .select_from(Image)\
                        .join(CroppedImage).join(Detector) \
                        .join(FaceImage).join(EmbeddingModel) \
                        .join(QualityImage).join(QualityModel) \
                        .filter(CroppedImage.face_detected == True,QualityImage.quality_group != None) \
                        .limit(2000000) \
                        .all()
                        #

im_cols = ['identity', 'pitch', 'angle_pitch', 'yaw', 'angle_yaw', 'roll','angle_roll',
           'source', 'gender', 'headgear',  'age','age_number',
            'glasses', 'type',  'beard', 'emotion', 'other_occlusions',
            'race',  'image_id']


df = pd.DataFrame([ [getattr(r[0],attr)   for attr in im_cols]  + list(r[1:5])
                   for r in results],
                   columns=im_cols + ['Detector','E_Model','Q_model','Q_group']  )

session.commit() 
df.to_pickle(os.path.join(output_dir,'df_test2.pkl'))




: 

: 

In [5]:
# query = (session.query(EnfsiPair).limit(100).all())

# for img in query:
#     filter = ['image_quality']
#     cat = img.get_category(im_category_list = filter,fi_cat_list=[],detector='mtcnn',embedding_model='Arcface')

#     cat2 = img.get_category([],[],'mtcnn','Arcface')

#     a = 5
